In [4]:
# Life Care Plan Costing Script

import pandas as pd
import os
import json

def compute_life_care_plan(
    report_date,
    life_expectancy,
    services,
    discount_rate
):
    """
    Computes detailed yearly costs for services in a Life Care Plan from 'report_date'
    through 'life_expectancy' using:
      - service-specific annual growth rates for cost escalation
      - user-defined discount_rate for present value calculations

    Returns:
        dfs: A dictionary of pandas DataFrames, one per service.
    """

    # Convert to Timestamp
    start_dt = pd.to_datetime(report_date)
    years_covered = int(life_expectancy)

    service_tables = {}

    for service_name, base_costs in services.items():
        # Prepare data dictionary for the service
        data = {
            "Year": [],
            "Age": [],
            "Portion of Year (%)": [],
            "Service": [],
            "Cost Components": [],
            "Total Annual Cost": [],
            "Present Value": [],
        }

        cumulative_present_value = 0

        for year in range(start_dt.year, start_dt.year + years_covered):
            age = year - start_dt.year
            portion_of_year = (
                1.0
                if year != start_dt.year
                else (365.25 - start_dt.day_of_year) / 365.25
            )

            total_cost = 0
            components_info = []

            for component, (base_cost, growth_rate) in base_costs.items():
                adjusted_cost = base_cost * ((1 + growth_rate) ** age)
                total_cost += adjusted_cost

                data["Year"].append(year)
                data["Age"].append(age)
                data["Portion of Year (%)"].append(f"{portion_of_year * 100:.2f}%")
                data["Service"].append(service_name)
                data["Cost Components"].append(f"{component}: ${adjusted_cost:,.2f}")
                data["Total Annual Cost"].append("")  # Placeholder for consistency
                data["Present Value"].append("")      # Placeholder for consistency

            years_from_report = year - start_dt.year + (1 - portion_of_year)
            present_value = total_cost / ((1 + discount_rate) ** years_from_report)
            cumulative_present_value += present_value

            # Append summary information
            data["Year"].append(year)
            data["Age"].append(age)
            data["Portion of Year (%)"].append("")
            data["Service"].append("Total")
            data["Cost Components"].append("")
            data["Total Annual Cost"].append(f"${total_cost:,.2f}")
            data["Present Value"].append(f"${present_value:,.2f}")

        # Convert to DataFrame
        df = pd.DataFrame(data)
        # Add cumulative present value as a separate row
        df.loc[len(df)] = ["", "", "", "Cumulative Present Value", "", "", f"${cumulative_present_value:,.2f}"]
        service_tables[service_name] = df

    return service_tables

def save_progress(data, filename="life_care_plan_progress.json"):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Progress saved to {filename}")

def load_progress(filename="life_care_plan_progress.json"):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            data = json.load(f)
        print(f"Progress loaded from {filename}")
        return data
    else:
        print(f"No saved progress found with filename {filename}. Starting fresh.")
        return {}

def main():
    print("Welcome to the Life Care Plan Costing Tool!")

    # Initialize or load progress
    progress = {}
    load_choice = input("Do you want to load a saved session? (y/n): ").strip().lower()
    if load_choice == 'y':
        progress = load_progress()
    else:
        progress = {}

    # 1. Gather user inputs
    if 'date_of_report' not in progress:
        progress['date_of_report'] = input("Enter the date of report (YYYY-MM-DD): ")
        save_progress(progress)
    date_of_report = progress['date_of_report']

    if 'date_of_birth' not in progress:
        progress['date_of_birth'] = input("Enter the date of birth (YYYY-MM-DD): ")
        save_progress(progress)
    date_of_birth = progress['date_of_birth']

    if 'life_expectancy' not in progress:
        try:
            progress['life_expectancy'] = int(input("Enter the life expectancy in years (e.g., 75): "))
            save_progress(progress)
        except ValueError:
            print("Invalid input. Please enter an integer for life expectancy.")
            return
    life_expectancy = progress['life_expectancy']

    services = progress.get('services', {})
    while True:
        add_service = input("Would you like to add a service? (y/n): ").strip().lower()
        if add_service == 'y':
            service_name = input("Enter the name of the service: ")
            try:
                num_components = int(input(f"How many components does {service_name} have? "))
            except ValueError:
                print("Invalid input. Please enter an integer for the number of components.")
                continue

            base_costs = {}
            for _ in range(num_components):
                component_name = input("Enter the component name: ")
                try:
                    base_cost = float(input(f"Enter the base annual cost ($) for {component_name}: "))
                    growth_rate = float(input(f"Enter the annual growth rate (%) for {component_name}: ")) / 100
                except ValueError:
                    print("Invalid input. Please enter numeric values for cost and growth rate.")
                    continue
                base_costs[component_name] = (base_cost, growth_rate)

            services[service_name] = base_costs
            progress['services'] = services
            save_progress(progress)
        elif add_service == 'n':
            break
        else:
            print("Please enter 'y' or 'n'.")

    if not services:
        print("No services added. Exiting the tool.")
        return

    if 'discount_rate' not in progress:
        try:
            progress['discount_rate'] = float(input("Enter the discount rate for present value (as a percentage, e.g., 5): ")) / 100
            save_progress(progress)
        except ValueError:
            print("Invalid input. Please enter a numeric value for discount rate.")
            return
    discount_rate_input = progress['discount_rate']

    # 2. Compute the Life Care Plan costs tables
    service_tables = compute_life_care_plan(
        report_date=date_of_report,
        life_expectancy=life_expectancy,
        services=services,
        discount_rate=discount_rate_input
    )

    # 3. Export to Excel
    file_name = "Life_Care_Plan_Cost_Report.xlsx"
    file_path = os.path.join(os.getcwd(), file_name)

    with pd.ExcelWriter(file_path) as writer:
        for service_name, df in service_tables.items():
            # Ensure sheet names are within Excel's limit of 31 characters
            sheet_name = service_name[:31]
            df.to_excel(writer, index=False, sheet_name=sheet_name)

    print(f"\nCalculation complete! Your file has been saved to: {file_path}")

    # Offer to reset progress
    reset_choice = input("Do you want to reset your saved progress? (y/n): ").strip().lower()
    if reset_choice == 'y':
        if os.path.exists("life_care_plan_progress.json"):
            os.remove("life_care_plan_progress.json")
            print("Saved progress has been reset.")
    else:
        print("Your progress has been saved and can be loaded next time.")

# Run the script if executed directly
if __name__ == "__main__":
    main()


Welcome to the Life Care Plan Costing Tool!
Progress saved to life_care_plan_progress.json
Progress saved to life_care_plan_progress.json
Progress saved to life_care_plan_progress.json
Invalid input. Please enter numeric values for cost and growth rate.
Invalid input. Please enter numeric values for cost and growth rate.
Invalid input. Please enter numeric values for cost and growth rate.
Invalid input. Please enter numeric values for cost and growth rate.
Invalid input. Please enter numeric values for cost and growth rate.
Progress saved to life_care_plan_progress.json
Progress saved to life_care_plan_progress.json

Calculation complete! Your file has been saved to: /Users/chrisskerritt/EconomicWorkbook/Life_Care_Plan_Cost_Report.xlsx
Your progress has been saved and can be loaded next time.
